# Download model

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

# model_path = Path('/remote/gpu03/schiller/ExecLLM/models/huggingface').joinpath('microsoft/Phi-3.5-MoE-instruct')
model_path = Path('/remote/gpu03/schiller/ExecLLM/models/huggingface').joinpath('microsoft/Phi-3.5-mini-instruct')
model_path.mkdir(parents=True, exist_ok=True)

# snapshot_download(repo_id="microsoft/Phi-3.5-MoE-instruct", local_dir=mistral_models_path)
snapshot_download(repo_id="microsoft/Phi-3.5-mini-instruct", local_dir=model_path)

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

LICENSE:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

SECURITY.md:   0%|          | 0.00/2.66k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

# Run

In [8]:
 torch.cuda.empty_cache()

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained( 
    # "/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-MoE-instruct",
    "/remote/gpu03/schiller/ExecLLM/models/huggingface/microsoft/Phi-3.5-mini-instruct",
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,
    quantization_config=quantization_config
)

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-MoE-instruct") 
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct") 

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [2]:
messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
] 

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
}

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])

/remote/gpu03/schiller/.conda/envs/ExecLLM/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


 To solve the linear equation 2x + 3 = 7, you need to isolate the variable x. Here's how you can do it step by step:


1. Subtract 3 from both sides of the equation to get rid of the constant term on the left side:

   2x + 3 - 3 = 7 - 3

   which simplifies to:

   2x = 4


2. Now, divide both sides of the equation by 2 to solve for x:

   2x / 2 = 4 / 2


3. This simplifies to:

   x = 2


So, the solution to the equation 2x + 3 = 7 is x = 2.
